# ch3 - c11
使用 meapsingle 中的數據研究**單親家庭對學生數學成績的影響**。 這些數據適用於 2000 年密西根州東南部的部分學校。

* math4：四年級數學成績及格率

* free：有免費午餐資格的百分比

* pctsgle：非已婚夫妻家庭中兒童的百分比

* lmedinc：家庭收入中位數的對數（以美元為單位）

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import wooldridge
df = wooldridge.data("meapsingle")

**(i) 執行數學成績（math4）對單親家庭比例（pctsgle）的簡單回歸，並按通常的格式報告結果。解釋斜率係數。單親家庭的效應是大還是小？**

$$\widehat{math4} = \widehat{\beta_{0}} + \widehat{\beta_{1}} \space pctsgle$$

In [2]:
# Y: math4 
# X: pctsgle
model=sm.OLS(df["math4"], sm.add_constant(df["pctsgle"])).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  math4   R-squared:                       0.380
Model:                            OLS   Adj. R-squared:                  0.377
Method:                 Least Squares   F-statistic:                     138.9
Date:                Sun, 15 Oct 2023   Prob (F-statistic):           2.54e-25
Time:                        18:35:21   Log-Likelihood:                -901.95
No. Observations:                 229   AIC:                             1808.
Df Residuals:                     227   BIC:                             1815.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         96.7704      1.597     60.603      0.0

    Ans.
    pctsgle 係數為-0.8329，單親家庭對數學成績(math4)有負面影響。但影響似乎很小，如果 pctsgle 增加 10%，則math4百分比預計會降低 0.08329%，這是一個很小的影響。

**(ii) 將變數 lmedinc 和 free 加入回歸模型。 pctsgle 的係數會發生什麼變化？ 解釋發生了什麼事。**

In [18]:
import statsmodels.api as sm
# Y: math4 
# X: pctsgle, free, lmedinc
X = df[["pctsgle","free","lmedinc"]]
model_2 = sm.OLS(df["math4"], sm.add_constant( X )).fit()
print(model_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  math4   R-squared:                       0.460
Model:                            OLS   Adj. R-squared:                  0.453
Method:                 Least Squares   F-statistic:                     63.85
Date:                Sun, 15 Oct 2023   Prob (F-statistic):           6.63e-30
Time:                        00:19:05   Log-Likelihood:                -886.08
No. Observations:                 229   AIC:                             1780.
Df Residuals:                     225   BIC:                             1794.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         51.7232     58.478      0.884      0.3

    Ans. 
    Pctsgle係數減少，最下面一行產生共線性的警語

**(iii) 求 lmedinc 和 free 之間的樣本相關性。它是否具有你預期的符號?**

In [3]:
df[["pctsgle","free","lmedinc"]].corr()

,pctsgle,free,lmedinc
pctsgle,1.000000,0.826357,-0.868636
free,0.826357,1.000000,-0.746970
lmedinc,-0.868636,-0.746970,1.000000


    Ans.
    lmedinc 和 free 之間的樣本相關性為 -0.74，這是預期的關係，因為隨著收入中位數的增加，免費午餐的資格就會減少。

**(iv) lmedinc 和 free 相關係數大，是否因此需要刪除變數。**

    Ans.
    不，因為 lmedinc 和 free 變數之間的高相關性，不會使單親家庭對學生表現的因果效應的確定更加困難。

**(v) 找出第 (ii) 部分迴歸中出現的每個解釋變數的變異數膨脹因子 (VIF)。 哪個變數的 VIF 最大？這個信息是否影響你用來研究單親家庭對數學表現的因果效應的模型？**

$$  
\begin{cases} 
\begin{aligned}
\widehat{pctsgle} &= \widehat{a_{0}} + \widehat{a_{1}} \space free \space \space + \widehat{a_{2}} \space lmedinc, \space {R^2}_1
 \\ \widehat{free} &= \widehat{b_{0}} + \widehat{b_{1}} \space pctsgle + \widehat{b_{2}} \space lmedinc, \space {R^2}_2
 \\ \widehat{lmedinc} &= \widehat{c_{0}} + \widehat{c_{1}} \space pctsgle+ \widehat{c_{2}} \space free, \space {R^2}_3
\end{aligned}
\end{cases} 
$$

In [8]:
# 輔助模型: X 變數輪流當 Y 跑回歸模型
# "pctsgle","free","lmedinc"
features = ["pctsgle","free","lmedinc"]
dfvif = pd.DataFrame(index=features, columns=["R2","VIF","Tolerance"])
for col in features:
    Y = df[col]
    X = df[["pctsgle","free","lmedinc"]].drop(col, axis=1)
    model = sm.OLS(Y, sm.add_constant( X )).fit()
    r2 = model.rsquared
    dfvif.loc[col] = [r2, 1/(1 - r2), 1 - r2]
dfvif

,R2,VIF,Tolerance
pctsgle,0.825814,5.740981,0.174186
free,0.686332,3.188079,0.313668
lmedinc,0.757212,4.118812,0.242788


    通過比較這三個變數，很明顯，"pctsgle" 這個變數具有最高VIF(5.74)。
    一般來說，一個常見的閾值是VIF大於10或者5。如果某個變數的VIF超過這個閾值，這可能表明共線性問題。但值得注意的是，這只是一個指導，而不是固定的規則。